In [33]:
import sys
from pathlib import Path
from pprint import pprint


In [34]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [35]:
from sqlmodel import Session, select
from api.events.models import EventModel
from api.db.session import engine


In [36]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(
        10)                 # limit the number of rows and change the order
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("--------------------------------")
    print(str(query))
    
    results = session.exec(query).all()
    pprint(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10
--------------------------------
SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1
[EventModel(description='', id=1, time=datetime.datetime(2025, 5, 24, 8, 3, 57, 825320, tzinfo=datetime.timezone.utc), page='/contact', updated_at=datetime.datetime(2025, 5, 24, 8, 3, 57, 825675, tzinfo=datetime.timezone.utc)),
 EventModel(description='', id=2, time=datetime.datetime(2025, 5, 24, 8, 3, 58, 245158, tzinfo=datetime.timezone.utc), page='/contact', updated_at=datetime.datetime(2025, 5, 24, 8, 3, 58, 245177, tzinfo=datetime.timezone.utc)),
 EventModel(description='', id=3, time=datetime.datetime(2025, 5, 24, 8, 3, 58, 290503, tzinfo=datetime.timezone.utc), page='/contact', updated_at=datetime.datetime(2025, 5, 24,

In [49]:
from timescaledb.hyperfunctions import time_bucket
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ["/about", "/contact", "/pages", "pricing"] # you can comment out the pages you don't want to see
    start = datetime.now(timezone.utc) - timedelta(hours = 1)
    finish = datetime.now(timezone.utc) + timedelta(hours = 1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()  # we can label it if we want using: .label("event_count")
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page,
        )
        .order_by(
            bucket,
            EventModel.page
            )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    #print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 5, 25, 0, 0, tzinfo=datetime.timezone.utc), '/about', 2412),
 (datetime.datetime(2025, 5, 25, 0, 0, tzinfo=datetime.timezone.utc), '/contact', 2435),
 (datetime.datetime(2025, 5, 25, 0, 0, tzinfo=datetime.timezone.utc), '/pages', 2617),
 (datetime.datetime(2025, 5, 25, 0, 0, tzinfo=datetime.timezone.utc), 'pricing', 2536)]
